In [1]:
import re
from pathlib import Path

import numpy as np
import pandas as pd


def extract_yyyymm(path: str) -> str:
    m = re.search(r"(20\d{2}(0[1-9]|1[0-2]))", Path(path).name)
    if not m:
        raise ValueError(f"파일명에서 YYYYMM(예: 202510)을 찾지 못했습니다: {path}")
    return m.group(1)


def process_bijung_sheet(df: pd.DataFrame) -> dict:
    """
    비중취합 시트 전처리 (파일 구조가 조금 달라도 최대한 안전하게 동작하도록 위치 기반 처리)
    반환: {'result': df_result, '채권': df_채권, '주식': df_주식}
    """

    # 1) 첫 1행 제거 후 인덱스 리셋
    df_drop = df.drop(index=df.index[:1])
    df_origin = df_drop.reset_index(drop=True)

    # 2) 컬럼명 설정 (두 번째 행을 헤더로 사용하는 구조 가정)
    df_origin.columns = ["Category1"] + df_origin.iloc[0, 1:].tolist()
    df_origin = df_origin.drop(df_origin.index[0]).reset_index(drop=True)

    # 3) 홀수열 이름 복사 (1,3,5...의 이름을 2,4,6...에 복사)
    column_list = list(df_origin.columns)
    for j in range(1, len(column_list), 2):
        if j + 1 < len(column_list):
            column_list[j + 1] = column_list[j]
    df_origin.columns = column_list

    # ------------------------------------------------------------
    # A) 전체가 null인 "첫 행" 위치 찾기 (안전: 위치 기반)
    # ------------------------------------------------------------
    row_null_mask = df_origin.isna().all(axis=1).to_numpy()  # 1D 보장
    if row_null_mask.any():
        first_null_row_pos = int(np.argmax(row_null_mask))   # True가 처음 나오는 위치
        df_result = df_origin.iloc[:first_null_row_pos].copy()
    else:
        df_result = df_origin.copy()

    # ------------------------------------------------------------
    # B) 전체가 null인 "첫 컬럼" 위치 찾기 (안전: 위치 기반)
    # ------------------------------------------------------------
    col_null_mask = df_result.isna().all(axis=0).to_numpy()  # 1D 보장
    if col_null_mask.any():
        first_null_col_pos = int(np.argmax(col_null_mask))
    else:
        first_null_col_pos = df_result.shape[1]

    # 4) 채권/주식 분리
    df_채권 = df_result.iloc[:, :first_null_col_pos].copy()
    df_주식_temp = df_result.iloc[:, first_null_col_pos:].copy()

    # ------------------------------------------------------------
    # C) 주식 데이터 시작 열 찾기 (전체 null 아닌 첫 컬럼)
    # ------------------------------------------------------------
    if df_주식_temp.shape[1] > 0:
        stock_has_data_mask = (~df_주식_temp.isna().all(axis=0)).to_numpy()  # 1D 보장
        if stock_has_data_mask.any():
            first_data_col_pos = int(np.argmax(stock_has_data_mask))
            df_주식 = df_주식_temp.iloc[:, first_data_col_pos:].copy()
        else:
            df_주식 = pd.DataFrame()
    else:
        df_주식 = pd.DataFrame()

    # 5) 주식 첫 컬럼 이름 변경
    if not df_주식.empty:
        df_주식 = df_주식.rename(columns={df_주식.columns[0]: "Category2"})

    return {"result": df_result, "채권": df_채권, "주식": df_주식}


def load_from_this_folder(
    sheet_name: str = "비중취합",
    pattern: str = "*_사학연금_포트폴리오_분석.xlsm",
    base_dir: Path | None = None,
) -> dict:
    """
    base_dir:
      - None이면 '이 코드를 실행하는 스크립트(.py)가 있는 폴더'를 사용
      - 노트북/인터랙티브 환경이면 현재 작업폴더(Path.cwd())를 사용
    """
    if base_dir is None:
        try:
            base_dir = Path(__file__).resolve().parent  # .py 기준
        except NameError:
            base_dir = Path.cwd()  # 노트북/인터랙티브 기준

    files = sorted(base_dir.glob(pattern))
    if not files:
        raise FileNotFoundError(f"'{base_dir}' 폴더에서 패턴 '{pattern}'에 맞는 파일을 찾지 못했습니다.")

    results = {}

    for fp in files:
        yyyymm = extract_yyyymm(str(fp))
        try:
            df = pd.read_excel(fp, sheet_name=sheet_name)
            results[yyyymm] = process_bijung_sheet(df)

            out = results[yyyymm]
            print(
                f"[OK] {fp.name} -> {yyyymm} | "
                f"result={out['result'].shape}, 채권={out['채권'].shape}, 주식={out['주식'].shape}"
            )
        except Exception as e:
            print(f"[FAIL] {fp.name}: {e}")

    return results


# =========================
# 실행
# =========================
results = load_from_this_folder(
    sheet_name="비중취합",
    pattern="*_사학연금_포트폴리오_분석.xlsm"
)

# =========================
# 사용 예시
# =========================
# df202510 = results["202510"]["result"]
# df202510_채권 = results["202510"]["채권"]
# df202510_주식 = results["202510"]["주식"]


[OK] 202512_사학연금_포트폴리오_분석.xlsm -> 202512 | result=(42, 93), 채권=(42, 15), 주식=(42, 77)


In [2]:
# results 생성 후, df2025xx / df2025xx_채권 / df2025xx_주식 형태로 자동 변수 생성
for yyyymm, out in results.items():
    globals()[f"df{yyyymm}"] = out["result"]
    globals()[f"df{yyyymm}_채권"] = out["채권"]
    globals()[f"df{yyyymm}_주식"] = out["주식"]

# 1. 채권형

In [3]:
# 처리할 채권/주식 데이터프레임 목록 가져오기
채권_dfs = [name for name in globals() 
            if name.startswith('df20') and name.endswith('_채권') 
            and len(name.split('_')[0]) == 8]  # df202307
채권_dfs.sort()

주식_dfs = [name for name in globals() 
            if name.startswith('df20') and name.endswith('_주식') 
            and len(name.split('_')[0]) == 8]  # df202307
주식_dfs.sort()

# 채권형 최종 데이터 생성
for df_name in 채권_dfs:
    df_채권 = globals()[df_name]
    
    category = df_채권.iloc[:, 0]
    result = []

    for i in range(1, df_채권.shape[1], 2):
        left_col = df_채권.columns[i]
        right_col = df_채권.columns[i+1]
        fund_code = left_col

        for idx, cat in enumerate(category):
            if isinstance(cat, str):
                result.append({
                    "펀드코드": fund_code,
                    "데이터ID(DATA_ID)": cat,
                    "비중(WGT)": df_채권.iloc[idx, i+1]
                })
            else:
                result.append({
                    "펀드코드": fund_code,
                    "데이터ID(DATA_ID)": df_채권.iloc[idx, i+1],
                    "비중(WGT)": df_채권.iloc[idx, i]
                })

    new_df_채권 = pd.DataFrame(result)
    new_df_채권 = new_df_채권.dropna(subset=['비중(WGT)'])
    
    # Mapping
    category_map = {
        '0-3Y': 'Fixed Income_만기',
        '3-5Y': 'Fixed Income_만기',
        '5-10Y': 'Fixed Income_만기',
        '>10Y': 'Fixed Income_만기',
        'AAA': 'Fixed Income_신용등급',
        'AA': 'Fixed Income_신용등급',
        'A': 'Fixed Income_신용등급',
        'BBB': 'Fixed Income_신용등급',
        '<BBB': 'Fixed Income_신용등급',
        'NR': 'Fixed Income_신용등급',
        
        'Treasury': 'Fixed Income_섹터',
        'Government': 'Fixed Income_섹터',
        'Corporate': 'Fixed Income_섹터',
        'Securitized': 'Fixed Income_섹터',
        
        'Developed': 'Fixed Income_Region1',
        'Emerging': 'Fixed Income_Region1',
        
        'North America': 'Fixed Income_Region2',
        'Europe': 'Fixed Income_Region2',
        'Japan': 'Fixed Income_Region2',
        'Asia Pacific': 'Fixed Income_Region2',
        'Latin America': 'Fixed Income_Region2',
        'Africa/Middle East': 'Fixed Income_Region2',
        'South Africa' : 'Fixed Income_Region2',
        
        'United States': 'Fixed Income_Country',
        'United Kingdom': 'Fixed Income_Country',
        'United Arab Emirates': 'Fixed Income_Country',
        'Japan': 'Fixed Income_Country',
        'China': 'Fixed Income_Country',
        'Canada': 'Fixed Income_Country',
        'Italy': 'Fixed Income_Country',
        'Australia': 'Fixed Income_Country',
        'South Korea': 'Fixed Income_Country',
        'France': 'Fixed Income_Country',
        'Supranational': 'Fixed Income_Country',
        'Switzerland': 'Fixed Income_Country',
        'Mexico': 'Fixed Income_Country',
        'Netherlands': 'Fixed Income_Country',
        'Spain': 'Fixed Income_Country',
        'Ireland': 'Fixed Income_Country',
        'Denmark': 'Fixed Income_Country',
        'New Zealand': 'Fixed Income_Country',
        'Indonesia': 'Fixed Income_Country',
        'Saudi Arabia': 'Fixed Income_Country',
        'Brazil': 'Fixed Income_Country',
        'Germany': 'Fixed Income_Country',
        'Belgium': 'Fixed Income_Country',
        'Romania': 'Fixed Income_Country',
        'Austria': 'Fixed Income_Country',
        'Finland': 'Fixed Income_Country',
        'Luxemburg': 'Fixed Income_Country',
        'EU': 'Fixed Income_Country',
        'Greece': 'Fixed Income_Country',
        'Hungary': 'Fixed Income_Country',
        'India': 'Fixed Income_Country',
        'Israel': 'Fixed Income_Country',
        'Latvia': 'Fixed Income_Country',
        'Poland': 'Fixed Income_Country',
        'Portugal': 'Fixed Income_Country',
        'Slovakia': 'Fixed Income_Country',
        'South Africa': 'Fixed Income_Country',
        'Taiwan': 'Fixed Income_Country',
        'Sweden': 'Fixed Income_Country',
        'Hong Kong': 'Fixed Income_Country',
        'Peru': 'Fixed Income_Country',
        'Uruguay': 'Fixed Income_Country',
        'Norway': 'Fixed Income_Country',
        'Korea': 'Fixed Income_Country',
        'Singapore': 'Fixed Income_Country',
        'Bermuda': 'Fixed Income_Country',
        'Poland': 'Fixed Income_Country',
        'Israel': 'Fixed Income_Country',
        'Luxembourg': 'Fixed Income_Country',
        'Greece': 'Fixed Income_Country',
        'Chile': 'Fixed Income_Country',
        'Colombia': 'Fixed Income_Country',
        'Iceland': 'Fixed Income_Country',
        'Cayman Islands': 'Fixed Income_Country',
        'Supranationals': 'Fixed Income_Country',
        'Croatia': 'Fixed Income_Country',
        'Thailand': 'Fixed Income_Country',
        'Philippines': 'Fixed Income_Country',
        'Mauritius': 'Fixed Income_Country',
        'Egypt': 'Fixed Income_Country',
        'Argentina': 'Fixed Income_Country',
        'Malaysia': 'Fixed Income_Country',
        'Kazakhstan': 'Fixed Income_Country',
        'Slovenia': 'Fixed Income_Country',
        'Vietnam': 'Fixed Income_Country',
        'Ghana': 'Fixed Income_Country',
        'Estonia': 'Fixed Income_Country',
        'Czech Republic': 'Fixed Income_Country',
        'S.Korea': 'Fixed Income_Country',
        'south korea': 'Fixed Income_Country',
        'Turkey': 'Fixed Income_Country',
    }

    new_df_채권['자산구분(ASSET_ID)'] = new_df_채권['데이터ID(DATA_ID)'].map(category_map)
    new_df_채권['자산구분(ASSET_ID)'] = new_df_채권['자산구분(ASSET_ID)'].fillna('기타_분류 없음')

    # 최종 컬럼 추가
    final_df = new_df_채권.copy()
    기준일 = df_name[2:8] + '01'  # df202307_채권 -> 20230701
    final_df['기준일'] = 기준일
    final_df['구분'] = 'FUND'
    final_df['기관코드'] = '1061'
    final_df['BM코드'] = ''
    final_df['데이터구분'] = 'FUND'

    final_columns = ['기준일','구분','기관코드','펀드코드','BM코드','자산구분(ASSET_ID)','데이터ID(DATA_ID)','데이터구분','비중(WGT)']
    final_df = final_df.reindex(columns=final_columns)

    # globals()에 최종 데이터프레임 저장
    globals()[f"final_{df_name}"] = final_df

    print(f"{df_name} -> final_{df_name} 생성 완료 (크기: {final_df.shape})")


df202512_채권 -> final_df202512_채권 생성 완료 (크기: (294, 9))


In [4]:
# final_로 시작하고 _채권으로 끝나는 DataFrame 이름 찾기
final_df_names_채권 = [name for name in globals() 
                  if name.startswith('final_df20') and name.endswith('_채권')]

final_df_names_채권.sort()  # 기준일 순서로 정렬

# 모든 final_df 불러와서 리스트에 담기
final_dfs_채권 = [globals()[name] for name in final_df_names_채권]

# 행 기준으로 합치기
merged_df_채권 = pd.concat(final_dfs_채권, ignore_index=True)

print(f"병합 완료: {merged_df_채권.shape}")

# 필요하면 globals()에 저장
globals()['merged_final_채권'] = merged_df_채권

병합 완료: (294, 9)


In [5]:
# Country와 Region2의 데이터가 'Japan'으로 동일하여 모두 Country로 반영되는 문제 해결
# 'Japan'인 행만 마스크 
mask = merged_df_채권['데이터ID(DATA_ID)'] == 'Japan'

# 펀드별 + 일자별로 묶어서 각 그룹의 첫 번째 행 인덱스만 뽑기
idx = (
    merged_df_채권[mask]
    .groupby(['펀드코드', '기준일'])
    .apply(lambda g: g.index.min())
)

# 첫 번째 행의 '자산구분(ASSET_ID)' 컬럼 값 변경
merged_df_채권.loc[idx.values, '자산구분(ASSET_ID)'] = 'Fixed Income_Region2'

C:\Users\user\AppData\Local\Temp\ipykernel_22356\1019295137.py:9: FutureWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  .apply(lambda g: g.index.min())


# 2. 주식형

In [6]:
# 처리할 주식형 데이터프레임 목록 가져오기
주식_dfs = [name for name in globals() 
            if name.startswith('df20') and name.endswith('_주식') 
            and len(name.split('_')[0]) == 8]  # df202307_주식 등
주식_dfs.sort()

# 주식형 최종 데이터 생성
for df_name in 주식_dfs:
    df_주식 = globals()[df_name]
    
    # 첫 번째 열은 Category2
    category = df_주식.iloc[:, 0]
    result = []

    # 2열씩 묶어서 처리
    for i in range(1, df_주식.shape[1], 2):
        left_col = df_주식.columns[i]
        right_col = df_주식.columns[i+1]
        fund_code = left_col

        for idx, cat in enumerate(category):
            if isinstance(cat, str):
                result.append({
                    "펀드코드": fund_code,
                    "데이터ID(DATA_ID)": cat,
                    "비중(WGT)": df_주식.iloc[idx, i+1]
                })
            else:
                result.append({
                    "펀드코드": fund_code,
                    "데이터ID(DATA_ID)": df_주식.iloc[idx, i+1],
                    "비중(WGT)": df_주식.iloc[idx, i]
                })

    # 데이터프레임 생성
    new_df_주식 = pd.DataFrame(result)

    # 비중(WGT) NaN 제거
    new_df_주식 = new_df_주식.dropna(subset=['비중(WGT)'])
    # 비중(WGT)가 0인 행 제거 (문자열으로 들어가 있을 경우'0'도 제거)
    new_df_주식 = new_df_주식[new_df_주식['비중(WGT)'] != 0]
    new_df_주식 = new_df_주식[new_df_주식['비중(WGT)'] != '0']

    # Mapping
    category_map_주식 = {
        'Consumer Discretionary': 'Equity_Sector',
        'Consumer Staples': 'Equity_Sector',
        'Energy': 'Equity_Sector',
        'Financials': 'Equity_Sector',
        'Health Care': 'Equity_Sector',
        'Industrials': 'Equity_Sector',
        'IT': 'Equity_Sector',
        'Materials': 'Equity_Sector',
        'Communication Services': 'Equity_Sector',
        'Utilities': 'Equity_Sector',
        'Real Estate': 'Equity_Sector',
        'Consumer Disc.': 'Equity_Sector',
        'Consumer Stap.': 'Equity_Sector',
        'Telco': 'Equity_Sector',
        
        'Developed': 'Equity_Region1',
        'Emerging': 'Equity_Region1',
               
        'Africa/Middle East': 'Equity_Region2',
        'Asia Pacific': 'Equity_Region2',
        'Europe': 'Equity_Region2',
        'Latin America': 'Equity_Region2',
        'North America': 'Equity_Region2',

        'USA': 'Equity_Country',
        'United States': 'Equity_Country',
        'United Kingdom': 'Equity_Country',
        'United Arab Emirates': 'Equity_Country',
        'United Arab Emirate': 'Equity_Country',
        'Japan': 'Equity_Country',
        'China': 'Equity_Country',
        'Canada': 'Equity_Country',
        'Italy': 'Equity_Country',
        'Australia': 'Equity_Country',
        'South Korea': 'Equity_Country',
        'France': 'Equity_Country',
        'Supranational': 'Equity_Country',
        'Switzerland': 'Equity_Country',
        'Mexico': 'Equity_Country',
        'Netherlands': 'Equity_Country',
        'Spain': 'Equity_Country',
        'Ireland': 'Equity_Country',
        'Denmark': 'Equity_Country',
        'New Zealand': 'Equity_Country',
        'Indonesia': 'Equity_Country',
        'Saudi Arabia': 'Equity_Country',
        'Brazil': 'Equity_Country',
        'Germany': 'Equity_Country',
        'Belgium': 'Equity_Country',
        'Romania': 'Equity_Country',
        'Austria': 'Equity_Country',
        'Finland': 'Equity_Country',
        'Luxemburg': 'Equity_Country',
        'Luxembourg': 'Equity_Country',
        'EU': 'Equity_Country',
        'Greece': 'Equity_Country',
        'Hungary': 'Equity_Country',
        'India': 'Equity_Country',
        'Israel': 'Equity_Country',
        'Latvia': 'Equity_Country',
        'Poland': 'Equity_Country',
        'Portugal': 'Equity_Country',
        'Slovakia': 'Equity_Country',
        'South Africa': 'Equity_Country',
        'Taiwan': 'Equity_Country',
        'Sweden': 'Equity_Country',
        'Hong Kong': 'Equity_Country',
        'Peru': 'Equity_Country',
        'Uruguay': 'Equity_Country',
        'Norway': 'Equity_Country',
        'Korea': 'Equity_Country',
        'Singapore': 'Equity_Country',
        'Bermuda': 'Equity_Country',
        'Poland': 'Equity_Country',
        'Israel': 'Equity_Country',
        'Luxembourg': 'Equity_Country',
        'Greece': 'Equity_Country',
        'Chile': 'Equity_Country',
        'Colombia': 'Equity_Country',
        'Iceland': 'Equity_Country',
        'Cayman Islands': 'Equity_Country',
        'Supranationals': 'Equity_Country',
        'Croatia': 'Equity_Country',
        'Thailand': 'Equity_Country',
        'Philippines': 'Equity_Country',
        'Mauritius': 'Equity_Country',
        'Egypt': 'Equity_Country',
        'Argentina': 'Equity_Country',
        'Malaysia': 'Equity_Country',
        'Kazakhstan': 'Equity_Country',
        'Slovenia': 'Equity_Country',
        'Vietnam': 'Equity_Country',
        'Ghana': 'Equity_Country', 
        'Netherland': 'Equity_Country',
        'Columbia': 'Equity_Country',
        'Pakistan': 'Equity_Country',
        'Zambia': 'Equity_Country',
        'Saudi Arabia  ': 'Equity_Country',
        'Saudi Arabia': 'Equity_Country',
        'Burkina Faso': 'Equity_Country',
        'south korea': 'Equity_Country',
        'Turkey': 'Equity_Country',
    }

    new_df_주식['자산구분(ASSET_ID)'] = new_df_주식['데이터ID(DATA_ID)'].map(category_map_주식)
    new_df_주식['자산구분(ASSET_ID)'] = new_df_주식['자산구분(ASSET_ID)'].fillna('기타_분류 없음')

    # 최종 컬럼 추가
    final_df = new_df_주식.copy()
    기준일 = df_name[2:8] + '01'  # df202307_주식 -> 20230701
    final_df['기준일'] = 기준일
    final_df['구분'] = 'FUND'
    final_df['기관코드'] = '1061'
    final_df['BM코드'] = ''
    final_df['데이터구분'] = 'FUND'

    final_columns = ['기준일','구분','기관코드','펀드코드','BM코드','자산구분(ASSET_ID)','데이터ID(DATA_ID)','데이터구분','비중(WGT)']
    final_df = final_df.reindex(columns=final_columns)

    # globals()에 최종 데이터프레임 저장
    globals()[f"final_{df_name}"] = final_df

    print(f"{df_name} -> final_{df_name} 생성 완료 (크기: {final_df.shape})")


df202512_주식 -> final_df202512_주식 생성 완료 (크기: (980, 9))


In [7]:
# final_로 시작하고 _주식으로 끝나는 DataFrame 이름 찾기
final_df_names_주식 = [name for name in globals() 
                  if name.startswith('final_df20') and name.endswith('_주식')]

final_df_names_주식.sort()  # 기준일 순서로 정렬

# 모든 final_df 불러와서 리스트에 담기
final_dfs_주식 = [globals()[name] for name in final_df_names_주식]

# 행 기준으로 합치기
merged_df_주식 = pd.concat(final_dfs_주식, ignore_index=True)

print(f"병합 완료: {merged_df_주식.shape}")

# 필요하면 globals()에 저장
globals()['merged_final_주식'] = merged_df_주식

병합 완료: (980, 9)


In [8]:
merged_df_주식.to_excel("주식_가공전.xlsx", index = False, header = True)

In [9]:
# 중복된 값 제거
# 1. Japan: Equity_country 2개 반영
# 2. 동일한 펀드 두개 들어있는 경우

# 완전 동일한 행(모든 컬럼 동일) 중 중복된 행만 보기
dupes = merged_df_주식[merged_df_주식.duplicated(keep=False)]

print("중복된 행(모든 컬럼 동일):")
print(dupes)
print(f"중복된 행 개수: {len(dupes)}")

중복된 행(모든 컬럼 동일):
          기준일    구분  기관코드          펀드코드 BM코드  자산구분(ASSET_ID) 데이터ID(DATA_ID)  \
167  20251201  FUND  1061  KRL232106111       Equity_Country          Japan   
172  20251201  FUND  1061  KRL232106111       Equity_Country          Japan   
288  20251201  FUND  1061  KRL107106110       Equity_Country          Japan   
290  20251201  FUND  1061  KRL107106110       Equity_Country          Japan   
410  20251201  FUND  1061  KRL213106115       Equity_Country          Japan   
..        ...   ...   ...           ...  ...             ...            ...   
847  20251201  FUND  1061  KRL203106108       Equity_Country      Hong Kong   
848  20251201  FUND  1061  KRL203106108       Equity_Country        Finland   
849  20251201  FUND  1061  KRL203106108       Equity_Country          Spain   
850  20251201  FUND  1061  KRL203106108       Equity_Country        Ireland   
851  20251201  FUND  1061  KRL203106108       Equity_Country          China   

    데이터구분   비중(WGT)  
167  FUND  5

In [10]:
# 모든 컬럼이 완전히 같은 행만 제거
duplicates_df_주식 = merged_df_주식.drop_duplicates()

In [11]:
# 중복 제거 전 행 수
before = len(merged_df_주식)

# 중복 제거
duplicates_df_주식 = merged_df_주식.drop_duplicates()

# 중복 제거 후 행 수
after = len(duplicates_df_주식)

print(f"삭제된 행 개수: {before - after}")

삭제된 행 개수: 43


In [12]:
# 중복행 제거 안된 행이 있을 경우



In [13]:
# Japan: Equity_region2 추가

# japan개수가 하나일 경우 
mask1_df = duplicates_df_주식.copy()

# (펀드코드, 기준일)로 그룹핑해서 Japan이 몇 개인지 확인
mask1 = (
    mask1_df['데이터ID(DATA_ID)'].eq('Japan') &
    mask1_df.groupby(['펀드코드','기준일'])['데이터ID(DATA_ID)'].transform(
        lambda x: (x=='Japan').sum() == 1
    )
)

# 조건에 맞는 행만 복제
rows_to_duplicate = mask1_df[mask1]

# 원본 df에 행 추가(인덱스 리셋)
mask1_df_주식 = pd.concat([mask1_df, rows_to_duplicate], ignore_index=True)

In [14]:
# 추가할 데이터 확인 
rows_to_duplicate

,기준일,구분,기관코드,펀드코드,BM코드,자산구분(ASSET_ID),데이터ID(DATA_ID),데이터구분,비중(WGT)
167,20251201,FUND,1061,KRL232106111,,Equity_Country,Japan,FUND,5.060000
288,20251201,FUND,1061,KRL107106110,,Equity_Country,Japan,FUND,5.873642
410,20251201,FUND,1061,KRL213106115,,Equity_Country,Japan,FUND,4.190000
790,20251201,FUND,1061,KRL301106117,,Equity_Country,Japan,FUND,5.012795


In [15]:
mask2 = mask1_df_주식['데이터ID(DATA_ID)'] == 'Japan'

idx = (
    mask1_df_주식[mask2]
    .groupby(['펀드코드', '기준일'])['데이터ID(DATA_ID)']  # 특정 컬럼 지정
    .apply(lambda g: g.index.min())
)

mask1_df_주식.loc[idx.values, '자산구분(ASSET_ID)'] = 'Equity_Region2'

In [16]:
mask1_df_주식 = mask1_df_주식.sort_index().reset_index(drop=True)

In [17]:
# 정렬
mask1_df_주식 = (
    mask1_df_주식
    .sort_values(['기준일', '펀드코드', '자산구분(ASSET_ID)'])
    .reset_index(drop=True)
)

In [18]:
# 'Japan'인 행만 중복 체크
mask_japan = mask1_df_주식['데이터ID(DATA_ID)'] == 'Japan'

# 동일한 (기준일,펀드코드,자산구분) 조합에서 'Japan'이 여러 개일 때
# 첫 번째는 제거(권역이랑 비중 동일), 그 이후 것들을 True로 표시
dup_idx = (
    mask1_df_주식[mask_japan]
    .duplicated(subset=['기준일','펀드코드','자산구분(ASSET_ID)'], keep = 'last')
)

# 원본 인덱스로 복원
idx_to_drop = mask1_df_주식[mask_japan].index[dup_idx]

# 해당 행 삭제
mask2_df_주식 = mask1_df_주식.drop(idx_to_drop).reset_index(drop=True)

In [19]:
mask2_df_주식

,기준일,구분,기관코드,펀드코드,BM코드,자산구분(ASSET_ID),데이터ID(DATA_ID),데이터구분,비중(WGT)
0,20251201,FUND,1061,KRL107106105,,Equity_Country,United States,FUND,58.920000
1,20251201,FUND,1061,KRL107106105,,Equity_Country,India,FUND,6.880000
2,20251201,FUND,1061,KRL107106105,,Equity_Country,China,FUND,6.090000
3,20251201,FUND,1061,KRL107106105,,Equity_Country,South Korea,FUND,5.200000
4,20251201,FUND,1061,KRL107106105,,Equity_Country,Denmark,FUND,4.200000
...,...,...,...,...,...,...,...,...,...
936,20251201,FUND,1061,KRL301106119,,Equity_Sector,Health Care,FUND,23.152226
937,20251201,FUND,1061,KRL301106119,,Equity_Sector,Industrials,FUND,12.046112
938,20251201,FUND,1061,KRL301106119,,Equity_Sector,IT,FUND,24.805726
939,20251201,FUND,1061,KRL301106119,,Equity_Sector,Materials,FUND,4.349175


# 3. 최종

In [20]:
# 두 개 병합 (행 기준)
merged_all_final = pd.concat([merged_df_채권, mask2_df_주식], ignore_index=True)

print(f"주식 + 채권 병합 완료: {merged_all_final.shape}")

# globals()에 저장
globals()['merged_final_주식_채권'] = merged_all_final

주식 + 채권 병합 완료: (1235, 9)


In [21]:
merged_all_final.to_excel("펀드 비중_사학연금.xlsx", index = False, header = True)

In [22]:
# 검증
# Japan 두개씩 입력되어 있는지 확인

# 기준일, 펀드코드별로 Japan 개수 세기
japan_counts = (
    merged_all_final[merged_all_final['데이터ID(DATA_ID)'] == 'Japan']  # DATA_ID가 Japan인 행만 선택
    .groupby(['기준일', '펀드코드'])                         # 기준일과 펀드코드로 그룹화
    .size()                                                 # 그룹별 행 개수 계산
    .reset_index(name='Japan_count')                        # 결과를 DataFrame으로 변환
)

display(
    japan_counts.style.set_table_attributes('style="max-height:400px; overflow-y: auto; display:block;"'))

,기준일,펀드코드,Japan_count
0,20251201,KRL107106105,2
1,20251201,KRL107106109,2
2,20251201,KRL107106110,2
3,20251201,KRL203106108,2
4,20251201,KRL210106105,2
5,20251201,KRL213106103,2
6,20251201,KRL213106112,2
7,20251201,KRL213106113,2
8,20251201,KRL213106114,2
9,20251201,KRL213106115,2


In [23]:
# 운용사 수신 시 Region2에는 있고 Country에 없는 경우

# Japan이 1개인 그룹 확인
japan_one = japan_counts[japan_counts['Japan_count'] == 1]

japan_one

,기준일,펀드코드,Japan_count
